In [1]:
from solana.rpc.api import Client
from solana.publickey import PublicKey
import base64
client = Client("https://extend.mainnet.rpcpool.com/30af34cbb7f1e44cf8776c7ecae8")

In [2]:
class EventQueueHeader: 
    def __init__(self, bytes):
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.head = int.from_bytes(bytes[8:16], "little")
        self.count = int.from_bytes(bytes[16:24], "little")
        self.seq_num = int.from_bytes(bytes[24:32], "little")
    def __str__(self):
        return (
            "Account Flags: " + str(self.account_flags) + "\n" +
            "Head: " + str(self.head) + "\n" +
            "Count: " + str(self.count) + "\n" + 
            "Sequence Number: " + str(self.seq_num) + "\n" 
        )

class Event:
    def __init__(self, bytes):
        self.event_flags = int.from_bytes(bytes[0:1], "little")
        self.owner_slot = int.from_bytes(bytes[1:2], "little")
        self.fee_tier = int.from_bytes(bytes[2:3], "little")
        padding = []
        for i in range(5):
            padding.append(int.from_bytes(bytes[3+i:4+i], "little"))
        self._padding = padding
        self.native_quantity_released = int.from_bytes(bytes[8:16], "little")
        self.native_quantity_paid = int.from_bytes(bytes[16:24], "little")
        self.native_fee_or_rebate = int.from_bytes(bytes[24:32], "little")
        self.order_id = int.from_bytes(bytes[32:48], "little")
        self.owner = PublicKey(bytes[48:80]).__str__()
        self.client_order_id = int.from_bytes(bytes[80:], "little")
    def __str__(self):
        return (
            "Event Flags: " + str(self.event_flags) + "\n" + 
            "Owner Slot: " + str(self.owner_slot) + "\n" + 
            "Fee Tier: " + str(self.fee_tier) + "\n" + 
            "Padding: " + ', '.join(str(item) for item in self._padding) + "\n" + 
            "Native Quantity Released: " + str(self.native_quantity_released) + "\n" +
            "Native Quantity Paid: " + str(self.native_quantity_paid) + "\n" + 
            "Native Fee or Rebate: " + str(self.native_fee_or_rebate) + "\n" + 
            "Order ID: " + str(self.native_fee_or_rebate) + "\n" + 
            "Owner: " + str(self.owner) + "\n" + 
            "Client Order ID: " + str(self.client_order_id) + "\n"
        )

class EventAccount:
    def __init__(self, bytes):
        bytes_without_padding = bytes[5:-7]
        header_bytes = bytes_without_padding[0:32]
        events_bytes = bytes_without_padding[32:]
        self.event_queue_header = EventQueueHeader(header_bytes)
        self.events = []
        for i in range(self.event_queue_header.count):
            starting_index = ((self.event_queue_header.head + i) % 2978) * 88
            ending_index = (starting_index + 88) 
            self.events.append(Event(
                events_bytes[starting_index:ending_index]
            ))
            
    def print_overview(self):
        print(self.event_queue_header)
        print("Number of Events", len(self.events))
        
    def print_events(self):
        for i in range(self.event_queue_header.count):
            print("------------ Event Number", i+1, "----------")
            print(self.events[i])

In [3]:
class OpenOrdersAccount: 
    def __init__(self, bytes):
        # Remove "serum" and "padding"
        bytes = bytes[5:-7]
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.market = []
        for i in range(32):
            self.market.append(int.from_bytes(bytes[8+i:9+i], "little"))
        self.owner = PublicKey(bytes[40:72]).__str__()
        self.native_base_free = int.from_bytes(bytes[72:80], "little")
        self.native_base_total = int.from_bytes(bytes[80:88], "little")
        self.native_quote_free = int.from_bytes(bytes[88:96], "little")
        self.native_quote_total = int.from_bytes(bytes[96:104], "little")
        self.free_slot_bits = int.from_bytes(bytes[104:120], "little")
        self.is_bid_bits = int.from_bytes(bytes[120:136], "little")
        self.orders = []
        for i in range(128):
            starting_index = 136 + i*16
            ending_index = 136 + (i + 1) * 16
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.client_order_ids = []
        for i in range(128):
            starting_index = 2184 + i*8
            ending_index = 2184 + (i + 1) * 8
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.referrer_rebates_accrued = int.from_bytes(bytes[3208:3216], "little")
        
    def __str__(self):
        return (
            "Accounts Flags: " + str(self.account_flags) + "\n" + 
            "Market: " + ', '.join(str(item) for item in self.market) + "\n" + 
            "Owner: " + str(self.owner) + "\n" + 
            "Native Base Free: " + str(self.native_base_free) + "\n" + 
            "Native Base Total: " + str(self.native_base_total) + "\n" + 
            "Native Quote Free: " + str(self.native_quote_free) + "\n" + 
            "Native Quote Total: " + str(self.native_quote_total) + "\n" + 
            "Free Slot Bits: " + str(self.free_slot_bits) + "\n" + 
            "Is Bid Bits: " + str(self.is_bid_bits) + "\n" + 
            "Orders: " + ', '.join(str(item) for item in self.orders) + "\n" + 
            "Client Order IDs: " + ', '.join(str(item) for item in self.client_order_ids) + "\n" +
            "Referer Rebates Accrued: " + str(self.referrer_rebates_accrued)
        )
    
    def get_owner(self):
        return self.owner
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

In [4]:
blocks = []
for block_num in range(137035226 - 100, 137035226 + 1):
    confirmed_block = client.get_confirmed_block(block_num)
    blocks.append(confirmed_block)

From parsing the two open orders accounts, CmFyxKyGG9x9jro7QqRAo6ejEg4QJNboqmgAvpCQ74hG and cwZmnbvcFr4nL7bhmqqcuxNzmKtdG6Te17ZLcRsj4xv, we know our's and Alameda's wallet addresses respectively: 5xoBq7f7CDgZwqHrDBdRWM84ExRetg4gZq93dyJtoSwp and CuieVDEDtLo7FypA9SbLM9saXFdb1dsshEkyErMqkRQq. We can filter the transactions in the blocks for these wallets. By analyzing the behaviour we want to determine the following

1. What is the through put of Alameda's orders? To our understanding not all fills/cancels make it to the chain and some are dropped (we weren't able to find some of Jump's cancels on the chain using sol scan)
2. Find the corresponding transactions for our trading behaviour. Look at our volume. 

In [5]:
# Taking a look at Alameda's behaviour
alameda_transactions = []
jump_transactions = []
both_transactions = []
both_transactions2 = []
for block in blocks:
    if 'result' not in block:
        continue
    transactions = block['result']['transactions']
    block_time = block['result']['blockTime']
    for transaction in transactions: 
        if '5xoBq7f7CDgZwqHrDBdRWM84ExRetg4gZq93dyJtoSwp' in transaction['transaction']['message']['accountKeys']:
            jump_transactions.append(transaction)
        if 'CuieVDEDtLo7FypA9SbLM9saXFdb1dsshEkyErMqkRQq' in transaction['transaction']['message']['accountKeys']:
            alameda_transactions.append(transaction)
        if 'cwZmnbvcFr4nL7bhmqqcuxNzmKtdG6Te17ZLcRsj4xv' in transaction['transaction']['message']['accountKeys'] and\
            'CuieVDEDtLo7FypA9SbLM9saXFdb1dsshEkyErMqkRQq' in transaction['transaction']['message']['accountKeys']:
            both_transactions.append(transaction)
        if '5xoBq7f7CDgZwqHrDBdRWM84ExRetg4gZq93dyJtoSwp' in transaction['transaction']['message']['accountKeys'] and\
            'CmFyxKyGG9x9jro7QqRAo6ejEg4QJNboqmgAvpCQ74hG' in transaction['transaction']['message']['accountKeys']:
            both_transactions2.append(transaction)

In [9]:
len(both_transactions)

103

In [10]:
len(both_transactions2)

118

In [8]:
len(both_transactions)

103

In [23]:
for transaction in both_transactions:
    print(transaction['transaction']['signatures'][0])

3yQ1aWUjdiwKxbfQeKrYnvQhscW1BKcfnV4X7xyTcZpCYiLy1VvYQQmM9oNDhynAjCbkCWhQjBHZr4gSqtRvzUQD
2hZtqqq99AkPkPmu9iytp8Bzzn2Bx8A2YThrwbb3DMj8gcTnKW5i1Ev1vGkizQVqJCG6vQkTiLuW6JLt6iUghcHd
25QhMsCCYUVYtufaGSEdpxtxtE5wBso8w1jRrYXZNLMqJSkDEt7VDy1LZWZcJaSaHGLtPkFUHYog6vPswdo9XWTb
3C3pwo8SS7nGaykZ5wa4N8uoxhnoG5PWLJYSVqCs85dpgw3E2ZnUXoQxxWd4WsbwRxxsVvtjcF1mTKdNz8HAWFth
65UphJ6d3QNc5MJcKanJeT84QYHAC1CVxRG19iHd9U5j3GZXSyDbkx7S7K9WYbdbU584YBhP7LbEDPr1JNfmTA31
2ght6XJ9WBj1WaSCrRTf23y5pgU1kpUjoesf9PwRLT7Q2UW7EbHt3F6pmcv2YwKgQg5PPFpufiRCXdCt8tttPkdP
4jv9mpPxb1RwWX8gNtvXPDauwsLr3hjw1WBk6qtSgLVzpkC7uH36WUSUoAuqyHHFbR9Pp3BdaAHiL4Jn3AtZ2ama
2qCJQt88LMa8hYnzZurAJwqdv6WEdgXHQBXdB5EeifKaG8kme2GsNPomo2azUYRxxga4UReBoSvWt84xoAQGsHFy
QCQH33UXhTkFEy14PRn44xVoMr7a46h9Xt7bohkgp9v7LQdNq1FPtuxKAbr41fKP4nHyzZUaZ7b1vWfcDnGiDBW
4KzWq5r18askQWJioyvFEQyeAGmHAuinJoKGTT4RoAAGj5ZVw5GWefm5i1M4yiTRCq9dujAsv38ZTjCY3f2BmBcR
3WtyhmFcM1DFwjKq3b3UZKHFD822TuJfNLLfgnxDiFNnov5R3QhZY33C8onGhGKUs5WzggYd2CepBtnPbMAQQsUw
5yPSDXB8woSG3muNFQAQB7

In [20]:
len(both_transactions2)

158